In [1]:
import pandas as pd
from flask import Flask,render_template,request,url_for
from flask_script import Manager
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
labels=[]
contents=[]
emailLabels=[]
app=Flask(__name__)
manager =Manager(app)


In [2]:
"""Split the contents and labels"""
with open('SMSSpamCollection','r') as f:
	filer=f.readlines()
	for line in filer:
		labels.append(line[:4].strip())
		contents.append(line[4:].strip())


In [3]:
"""Split train test data in ratio of 80:20"""
labels_train,labels_test,contents_train,contents_test=train_test_split(labels,contents,test_size=0.2,random_state=42)

In [7]:
""" Create dictionary containing the frequency of each word in the data """
count=CountVectorizer()
count_cross=CountVectorizer()

sms_transform=count.fit_transform(contents_train)
sms_test=count.transform(contents_test)
print(len(count.vocabulary_))

#cross_val_score_transforamtion
sms_transform_cross=count_cross.fit_transform(contents)
count.vocabulary_


7726


{'partnership': 5078,
 'po': 5258,
 'price': 5397,
 'spontaneously': 6383,
 'babysit': 1199,
 '87021': 697,
 'times': 6879,
 '82242': 663,
 '500': 542,
 'linerental': 4106,
 'unbelievable': 7112,
 'pride': 5399,
 'alcohol': 905,
 'youuuuu': 7700,
 'rude': 5832,
 'films': 2826,
 'box385': 1471,
 'brisk': 1516,
 'pocketbabe': 5272,
 '83383': 674,
 'mobilesdirect': 4536,
 'moms': 4554,
 'taste': 6694,
 'over': 5009,
 'hang': 3304,
 'footbl': 2917,
 'greeting': 3230,
 '2morow': 407,
 'foot': 2915,
 'figure': 2816,
 'nb': 4703,
 'taught': 6699,
 'grazed': 3223,
 'superb': 6592,
 'deciding': 2199,
 'bong': 1431,
 'tenerife': 6747,
 'friendships': 2991,
 'witin': 7537,
 'avble': 1161,
 'savamob': 5907,
 'surfing': 6610,
 'opinions': 4943,
 'pei': 5124,
 'thinks': 6825,
 '50s': 553,
 'seven': 6027,
 'department': 2247,
 'celebrations': 1711,
 '4thnov': 536,
 'alrite': 933,
 'stock': 6464,
 'mnths': 4529,
 'realise': 5588,
 'goals': 3153,
 'some': 6277,
 '07099833605': 23,
 '09066362231': 234,


In [8]:
""" Create a document matrix of each word in each document """
features=TfidfTransformer()
features_cross=TfidfTransformer()

feat=features.fit_transform(sms_transform)
contents1=features.transform(sms_test)

#TfidfTransformer_cross_val_transform
feat_cross=features_cross.fit_transform(sms_transform_cross)

In [9]:
"""Create SVM classifier"""
clf=svm.LinearSVC(C=10000.0)#,kernel="rbf",gamma=0.6)
clf.fit(feat,labels_train)
pred=clf.predict(contents1)

#cross_val_score_svm

clf_cross_val=svm.LinearSVC(C=10000.0)
cross_matrix = cross_val_score(clf_cross_val, feat_cross, labels)


In [10]:
print("Accuracy Score by Cross Validation: %s"%cross_matrix)

Accuracy Score by Cross Validation: [ 0.98170075  0.98223897  0.98062433]


In [11]:
acc=accuracy_score(labels_test,pred)

print ("Accuracy score by train_test_split: %s"%acc)
print("Average Accuracy score of Cross Validation %s"%(sum(cross_matrix)/len(cross_matrix)))


Accuracy score by train_test_split: 0.987443946188
Average Accuracy score of Cross Validation 0.981521349121


In [12]:
"""Confusin Matrix"""

print(confusion_matrix(pred, labels_test))
#C[0,0] contains True negatives i.e. True Not Spam detected as Not Spam
#C[0,1] contains False Positives i.e. True Not Spam detected as Spam
#C[1,0] contains False Negatives i.e. True Spam detected as Not Spam
#C[1,1] Contains False Positives i.e. True Spam deteceted as Spam


[[951  11]
 [  3 150]]
